In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

# import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

import matplotlib.patheffects as PathEffects

### load lidar xarray

In [2]:
# fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc' #ARS
fname = '~/Documents/research/sierra/data/compiled_SUPERsnow.nc' #BSU
# fname = '~/Documents/research/sierra/data/20m_analysis/compiled_SUPERsnow_20m.nc' #BSU
#~~~~ ds full
ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()

#~~~~ ds peak
dpeak = ds.isel(time=[0,7,18,30,42,49]) #0,7,22,28,41,49]
dpeak.close()

ds = dpeak
ds.close()

### load terrain xarray

In [3]:
# path = '/home/meganmason/Documents/projects/thesis/data/processing_lidar/depths_3m/equal_extent/terrain/*.nc' #ARS
path = '/Users/meganmason491/Documents/research/sierra/data/terrain/*.nc' #BSU
fpath = glob.glob(path)
fpath


['/Users/meganmason491/Documents/research/sierra/data/terrain/aspect.nc',
 '/Users/meganmason491/Documents/research/sierra/data/terrain/hillshade.nc',
 '/Users/meganmason491/Documents/research/sierra/data/terrain/slope.nc']

In [4]:
terrain=xr.open_mfdataset(fpath, concat_dim=None, combine='nested', chunks={'x':1000, 'y':1000}, parallel=True).rename({'Band1':'hillshade'}).drop('transverse_mercator') #combine='nested', 
terrain.close()
terrain

<xarray.Dataset>
Dimensions:    (x: 17569, y: 17002)
Coordinates:
  * x          (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y          (y) float64 4.179e+06 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
Data variables:
    aspect     (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    hillshade  (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    slope      (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 2.2.3, released 2017/11/20
    history:      Fri Oct  4 11:46:25 2019: ncrename -v Band1,aspect aspect.n...
    NCO:          4.7.2

### flatten

In [ ]:
#terrain
dem_flat = ds.dem.values.flatten()
asp_flat = terrain.aspect.values.flatten() # subtract 45 degrees so range is -45:345 (i.e N[-45:45], E[46:])
slp_flat = terrain.slope.values.flatten()

In [ ]:
dem_flat = dem_flat[::100000] #100,000 of 300,000,000 values = .001% of data
asp_flat = asp_flat[::100000]
slp_flat = slp_flat[::100000]

In [ ]:
dem_flat = np.where(dem_flat>0, dem_flat, np.nan)

In [ ]:
print(dem_flat.shape)
print(asp_flat.shape)
print(slp_flat.shape)

In [ ]:
fig, [ax1,ax2,ax3] = plt.subplots(1,3, figsize=(15,5))
ax1.hist(dem_flat, bins=25, rwidth=0.8,  label='elevation'); ax1.set_title('Elevation (m)')
ax2.hist(asp_flat, bins=([0,90,180,270,360]), rwidth=0.8,  label='aspect'); ax2.set_title('Aspect (deg)')
ax3.hist(slp_flat[slp_flat>0], bins=25, rwidth=0.8,  label='slope'); ax3.set_title('Slope (deg)')

In [ ]:
# ax2.hist(asp_flat, bins=([315, 0,90,180,270,360]), rwidth=0.8,  label='aspect'); ax2.set_title('Aspect (deg)')

# n = np.where((asp_flat>315) & (asp_flat<=45))
# n = np.where(terrain.aspect.values>315)# & (terrain.aspect.values<=45))


n = np.where((terrain.aspect>315) and (terrain.aspect<45)) #& terrain.aspect.where(terrain.aspect>315) 

In [ ]:
n

In [ ]:
plt.imshow(terrain.aspect)
plt.colorbar()

In [ ]:
terrain

In [ ]:
aspect=np.flip(terrain.aspect,0)
aspect=aspect.where(ds.mask==1)
aspect=aspect.to_dataset()
aspect.close()

In [ ]:
# plt.imshow(aspect)

In [ ]:
aspect

In [ ]:
np.random.seed(19680801)

# Compute pie slices
N = 8
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
print('theta size', len(theta))
#theta = np.array([0, np.pi / 2, np.pi, 3 * np.pi / 2, 2 * np.pi])
N = len(theta)
radii = 10 * np.random.rand(N)
print(max(theta))
print(radii.max())
width = 2* np.pi / N
colors = plt.cm.viridis(radii / 10.)

ax = plt.subplot(111, projection='polar')
ax.bar(theta, radii, width=width, bottom=0.0, color=colors, alpha=0.5)
ax.set_theta_zero_location("N")

plt.show()

In [ ]:
# data
yrly_means = ds.snow.groupby('time.year').mean(dim=('x','y')) # snow values grouped by year, then the mean
yrly_means = yrly_means.values.flatten()

print(yrly_means.shape)

# yrly_std = ds.snow.groupby('time.year').std('time')
# yrly_std = yrly_means.values.flatten()

# print(yrly_std.shape)

In [ ]:
plt.plot(ds.time, yrly_means, marker= 'x')

In [ ]:
d = []
for layer in list(ds.snow.groupby('time.year')): #.groups:
    print('grouped item', layer)
    
    

### plot

In [ ]:
snow=ds.snow.values
print(snow.shape)
# s=snow.values[::10000]
# s

In [ ]:
ds

In [ ]:
for year in ds.groupby('time.year'):
    a=ds.snow.values.flatten()
    a=a[::100000]
    print(a.shape)
    b=ds.dem.values.flatten()
    b=a[::100000]
    print(b.shape)
#     plt.hexbin(b,a,mincnt=1)
#     plt.show

In [ ]:

# fig = plt.figure(figsize=(15, 10))
# plt.hexbin(dem_flat, yrly_means, mincnt=1)
# plt.xlabel('Elevation [m]')
# plt.ylabel('$\sigma$')
# plt.title('Snow depth [50m] STD by elevation')
# plt.colorbar()
# plt.show()
# plt.savefig('figs/elevation_density_13-16', dpi=600)

In [ ]:
a=ds.snow[0].values.flatten()
a=a[::100000]
print(a.shape)
a[a<=0]=np.nan
# b=ds.dem.values.flatten()
# b=a[::100000]
# print(b.shape)

In [ ]:
plt.hexbin(dem_flat, a>0, mincnt=1)

In [ ]:
print(np.nanmin(dem_flat))